<a href="https://colab.research.google.com/github/ahimsaj/ML-Training/blob/main/ahimsa_friend_ratings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("/friends_train_ratings.csv")

In [3]:
train

,title,plot,ratings,date,total_reviews
0,The One Where Monica Gets a Roommate,"Monica and the gang introduce Rachel to the ""r...",8.3,22 Sep. 1994,"-8,583"
1,The One with the Sonogram at the End,Ross finds out his ex-wife is pregnant. Rachel...,8.0,29 Sep. 1994,"-6,607"
2,The One with the Thumb,Monica becomes irritated when everyone likes h...,8.1,6 Oct. 1994,"-6,213"
3,The One with George Stephanopoulos,Joey and Chandler take Ross to a hockey game t...,8.1,13 Oct. 1994,"-6,035"
4,The One with the East German Laundry Detergent,"Eager to spend time with Rachel, Ross pretends...",8.4,20 Oct. 1994,"-6,029"
...,...,...,...,...,...
190,The One with the Cooking Class,Monica's restaurant get a bad review. After co...,8.1,02-May-02,"-3,688"
191,The One Where Rachel Is Late,Monica and Phoebe bet on when Rachel's baby wi...,8.3,09-May-02,"-3,645"
192,The One Where Rachel Has a Baby: Part 1,Ross and Rachel go to the hospital to have the...,8.8,16-May-02,"-3,981"
193,The One Where Rachel Has a Baby: Part 2,"After Rachel gives birth to her baby, she must...",8.9,16-May-02,"-4,293"


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          195 non-null    object 
 1   plot           195 non-null    object 
 2   ratings        195 non-null    float64
 3   date           195 non-null    object 
 4   total_reviews  195 non-null    object 
dtypes: float64(1), object(4)
memory usage: 7.7+ KB


Ratings feature

In [5]:
px.histogram(train,x="ratings",nbins=20,title="Ratings of episodes in the training dataset")

In [6]:
train["date"] = pd.to_datetime(train["date"])

In [7]:
px.line(train,x="date",y="ratings",title="Ratings across different dates")

Observations:
Friends has an average rating of more than 8. The lowest rating is of a 7, which is not bad at all!

I could not find a pattern in the ratings over the years, the show seems to be consistently doing well.

Total reviews feature¶

In [8]:
px.line(train,y="total_reviews",x="date",title="Number of reviews for episodes in the training dataset")

In [9]:
length = []
for i in train["plot"]:
    length.append(len(i))
train["plot_length"] = length
px.bar(train,x="title",y="plot_length",title="Plot lengths")

Baseline Model

In [10]:
y = train["ratings"]
X = train["plot"]

In [11]:
#Necessary Imports
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier, XGBRegressor
param_grid = {
    'model__max_depth': [2, 3, 5, 7, 10],
    'model__n_estimators': [10, 100, 500],
}


pipe = Pipeline([('tfidf', TfidfVectorizer()), ('model', XGBRegressor())])
grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, scoring='neg_root_mean_squared_error')
grid.fit(X,y)

[13:54:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('model', XGBRegressor())]),
             n_jobs=-1,
             param_grid={'model__max_depth': [2, 3, 5, 7, 10],
                         'model__n_estimators': [10, 100, 500]},
             scoring='neg_root_mean_squared_error')

In [12]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

In [14]:
#Predicting
test = pd.read_csv("/friends_test_file.csv")

In [15]:
preds = grid.predict(test["plot"])

In [16]:
len(preds)

35

In [17]:
test.shape

(35, 7)

In [18]:
preds

array([8.634297, 8.320085, 8.351128, 8.442602, 8.516853, 8.267387,
       8.468478, 8.579031, 8.456208, 8.262337, 8.248636, 8.648726,
       8.244775, 8.354373, 8.327822, 8.213826, 8.374323, 8.825821,
       8.674115, 8.530194, 8.535954, 8.311661, 8.008631, 8.713164,
       8.541395, 8.664354, 8.409912, 8.857148, 8.43597 , 8.697681,
       8.443368, 8.137261, 8.58795 , 8.542828, 8.354012], dtype=float32)

In [19]:
test = test.reset_index()

In [20]:
ID =[]
for i in range(1,36):
    ID.append(i)

In [21]:
predictions = pd.DataFrame()
predictions["ID"]=ID
predictions["ratings"] =preds
predictions.to_csv("submission.csv",index=False)
predictions

,ID,ratings
0,1,8.634297
1,2,8.320085
2,3,8.351128
3,4,8.442602
4,5,8.516853
5,6,8.267387
6,7,8.468478
7,8,8.579031
8,9,8.456208
9,10,8.262337
